# Example 1 - Default Approach

This notebook demonstrates how to use the TX2 dashboard with a sequence classification transformer using the default approach as described in the "Basic Usage" docs.

In [1]:
%cd -q ..
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
os.environ['LRU_CACHE_CAPACITY'] = '1' 

We enable logging to view the output from `wrapper.prepare()` further down in the notebook. (It's a long running function, and logs which step it's on.)

In [3]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [4]:
import numpy as np
import pandas as pd
import torch
from torch import cuda
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoTokenizer, BertTokenizer

In this example notebook, we use the 20 newsgroups dataset, which can be downloaded through huggingfacevia below:

In [5]:
from datasets import load_dataset

# getting newsgroups data from huggingface
train_data = pd.DataFrame(data=load_dataset("SetFit/20_newsgroups", split="train"))
test_data = pd.DataFrame(data=load_dataset("SetFit/20_newsgroups", split="test"))

# setting up pytorch device
if cuda.is_available():
    device = "cuda"
elif torch.has_mps:
    device = "mps"
else:
    device = "cpu"

device

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/81n/.cache/huggingface/datasets/SetFit___json/SetFit--20_newsgroups-f05bfc706e284479/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


Found cached dataset json (/home/81n/.cache/huggingface/datasets/SetFit___json/SetFit--20_newsgroups-f05bfc706e284479/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


'cuda'

Defined below is a simple sequence classification model with a variable for the language model itself `l1`. Since it is a BERT model, we take the sequence embedding from the `[CLS]` token (via `output_1[0][:, 0, :])`) and pipe that into the linear layer.

In [6]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained("bert-base-cased")
        self.l2 = torch.nn.Linear(768, 20)

    def forward(self, ids, mask):
        output_1 = self.l1(ids, mask)
        output = self.l2(output_1[0][:, 0, :])  # use just the [CLS] output embedding
        return output


model = BERTClass()
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Some simplistic data cleaning, and putting all data into dataframes for the wrapper

In [7]:
def clean_text(text):
    text = str(text)
    # text = text[text.index("\n\n") + 2 :]
    text = text.replace("\n", " ")
    text = text.replace("    ", " ")
    text = text.replace("   ", " ")
    text = text.replace("  ", " ")
    text = text.strip()
    return text

In [8]:
# clean long white space or extensive character returns
train_data.text = train_data.text.apply(lambda x: clean_text(x))
test_data.text = test_data.text.apply(lambda x: clean_text(x))

# remove empty entries or trivially short ones
train_cleaned = train_data[train_data["text"].str.len() > 1]
test_cleaned = test_data[test_data["text"].str.len() > 1]
train_cleaned

,text,label,label_text
0,I was wondering if anyone out there could enli...,7,rec.autos
1,A fair number of brave souls who upgraded thei...,4,comp.sys.mac.hardware
2,"well folks, my mac plus finally gave up the gh...",4,comp.sys.mac.hardware
3,Do you have Weitek's address/phone number? I'd...,1,comp.graphics
4,"From article <C5owCB.n3p@world.std.com>, by to...",14,sci.space
...,...,...,...
11309,DN> From: nyeda@cnsvax.uwec.edu (David Nye) DN...,13,sci.med
11310,"I have a (very old) Mac 512k and a Mac Plus, b...",4,comp.sys.mac.hardware
11311,I just installed a DX2-66 CPU in a clone mothe...,3,comp.sys.ibm.pc.hardware
11312,Wouldn't this require a hyper-sphere. In 3-spa...,1,comp.graphics


## Training

This section minimally trains the classification and language model - nothing fancy here, just to give the dashboard demo something to work with. Most of this is similar to the huggingface tutorial notebooks.

In [9]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-05)

In [10]:
class EncodedSet(Dataset):
    def __init__(self, dataframe: pd.DataFrame, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        print(self.len)

    def __getitem__(self, index):
        text = str(self.data.text[index])
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_token_type_ids=True,
        )
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "targets": torch.tensor(self.data.label[index], dtype=torch.long),
        }

    def __len__(self):
        return self.len


train_cleaned.reset_index(drop=True, inplace=True)
test_cleaned.reset_index(drop=True, inplace=True)

train_set = EncodedSet(train_cleaned, tokenizer, 256)
test_set = EncodedSet(test_cleaned[:1000], tokenizer, 256)

train_params = {"batch_size": 16, "shuffle": True, "num_workers": 0}

test_params = {"batch_size": 2, "shuffle": True, "num_workers": 0}

# put everything into data loaders
train_loader = DataLoader(train_set, **train_params)
test_loader = DataLoader(test_set, **test_params)

11014
1000


In [11]:
def train(epoch):
    model.train()

    loss_history = []
    for _, data in tqdm(
        enumerate(train_loader, start=0), total=len(train_loader), desc=f"Epoch {epoch}"
    ):
        ids = data["ids"].to(device, dtype=torch.long)
        mask = data["mask"].to(device, dtype=torch.long)
        targets = data["targets"].to(device, dtype=torch.long)

        outputs = model(ids, mask).squeeze()

        optimizer.zero_grad()
        loss = loss_function(outputs, targets)
        if _ % 100 == 0:
            print(f"Epoch: {epoch}, Loss:  {loss.item()}")
        loss_history.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #         torch.cuda.empty_cache()
    return loss_history

In [12]:
losses = []
for epoch in range(1):
    losses.extend(train(epoch))


Epoch 0:   0%|          | 0/689 [00:00<?, ?it/s]

Epoch: 0, Loss:  3.139150381088257
Epoch: 0, Loss:  2.0730278491973877
Epoch: 0, Loss:  1.1646325588226318
Epoch: 0, Loss:  0.8112739324569702
Epoch: 0, Loss:  1.2353179454803467
Epoch: 0, Loss:  1.0301554203033447
Epoch: 0, Loss:  1.0289490222930908


The wrapper uses an `encodings` dictionary for various labels/visualizations, and can be set up with something similar to:

In [13]:
encodings = (
    train_cleaned[["label", "label_text"]]
    .groupby(["label_text"])
    .apply(lambda x: x["label"].tolist()[0])
    .to_dict()
)
encodings

{'alt.atheism': 0,
 'comp.graphics': 1,
 'comp.os.ms-windows.misc': 2,
 'comp.sys.ibm.pc.hardware': 3,
 'comp.sys.mac.hardware': 4,
 'comp.windows.x': 5,
 'misc.forsale': 6,
 'rec.autos': 7,
 'rec.motorcycles': 8,
 'rec.sport.baseball': 9,
 'rec.sport.hockey': 10,
 'sci.crypt': 11,
 'sci.electronics': 12,
 'sci.med': 13,
 'sci.space': 14,
 'soc.religion.christian': 15,
 'talk.politics.guns': 16,
 'talk.politics.mideast': 17,
 'talk.politics.misc': 18,
 'talk.religion.misc': 19}

## TX2

This section shows how to put everything into the TX2 wrapper to get the dashboard widget displayed.

In [14]:
from tx2.dashboard import Dashboard
from tx2.wrapper import Wrapper

/home/81n/micromamba/envs/tx2/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/81n/micromamba/envs/tx2/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/81n/micromamba/envs/tx2/lib/python3.9/site-packages/umap/distances.py:1086: 

In [15]:
model.eval() # shouldn't be necessary since done in wrapper

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [16]:
wrapper = Wrapper(
    train_texts=train_cleaned.text,
    train_labels=train_cleaned.label,
    test_texts=test_cleaned.text[:2000],
    test_labels=test_cleaned.label[:2000],
    encodings=encodings,
    classifier=model,
    language_model=model.l1,
    tokenizer=tokenizer,
    overwrite=True,
)
wrapper.batch_size = 16
wrapper.prepare()

INFO:root:Cache path found
INFO:root:Checking for cached predictions...
INFO:root:Running classifier...
INFO:root:Saving predictions...
INFO:root:Writing to data/predictions.json
INFO:root:Done!
INFO:root:Checking for cached embeddings...
INFO:root:Embedding training and testing datasets
INFO:root:Saving embeddings...
INFO:root:Writing to data/embedding_training.json
INFO:root:Writing to data/embedding_testing.json
INFO:root:Done!
INFO:root:Checking for cached projections...
INFO:root:Training projector...
INFO:root:Applying projector to test dataset...
INFO:root:Saving projections...
INFO:root:Writing to data/projections_training.json
INFO:root:Writing to data/projections_testing.json
INFO:root:Writing to data/projector.pkl.gz
INFO:root:Done!
INFO:root:Checking for cached salience maps...
INFO:root:Computing salience maps...


  0%|          | 0/2000 [00:00<?, ?it/s]

INFO:root:Saving salience maps...
INFO:root:Writing to data/salience.pkl.gz
INFO:root:Done!
INFO:root:Checking for cached cluster profiles...
INFO:root:Clustering projections...
INFO:root:Saving cluster profiles...
INFO:root:Writing to data/clusters.json
INFO:root:Writing to data/cluster_profiles.pkl.gz
INFO:root:Done!
INFO:root:Saving cluster labels...
INFO:root:Writing to data/cluster_labels.json
INFO:root:Done!
INFO:root:Saving cluster word counts...
INFO:root:Writing to data/cluster_words.json
INFO:root:Writing to data/cluster_class_words.json
INFO:root:Done!


In [17]:
# wrapper._compute_all_salience_maps_raw()

In [18]:
dash = Dashboard(wrapper)
dash.render()

To play with different UMAP and DBSCAN arguments without having to recompute the entire `prepare()` function, you can use `recompute_projections` (which will recompute both the projections and visual clusterings) or `recompute_visual_clusterings` (which will only redo the clustering)

In [19]:
wrapper.recompute_visual_clusterings("KMeans", clustering_args=dict(n_clusters=20))
# wrapper.recompute_visual_clusterings("OPTICS", clustering_args=dict())
# wrapper.recompute_projections(umap_args=dict(min_dist=.2), dbscan_args=dict())

INFO:root:Clustering projections...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO:root:Saving cluster profiles...
INFO:root:Writing to data/clusters.json
INFO:root:Writing to data/cluster_profiles.pkl.gz
INFO:root:Done!
INFO:root:Saving cluster labels...
INFO:root:Writing to data/cluster_labels.json
INFO:root:Done!
INFO:root:Saving cluster word counts...
INFO:root:Writing to data/cluster_words.json
INFO:root:Writing to data/cluster_class_words.json
INFO:root:Done!


To test or debug the classification model/see what raw outputs the viusualizations are getting, or create your own visualization tools, you can manually call the `classify()`, `soft_classify()`, `embed()` functions, or get access to any of the cached data as seen in the bottom cell

In [20]:
wrapper.classify(["testing"])

[8]

In [21]:
wrapper.soft_classify(["testing"])

[[-1.170468807220459,
  -0.42441028356552124,
  -0.4678337275981903,
  -0.7580161094665527,
  0.07608048617839813,
  -0.31217095255851746,
  0.48881950974464417,
  1.4150304794311523,
  1.7861741781234741,
  -0.1740753948688507,
  -0.19095690548419952,
  -0.34646695852279663,
  0.7207998633384705,
  -0.689913809299469,
  0.7936819791793823,
  -1.1157079935073853,
  -0.25155022740364075,
  -0.8481627106666565,
  -1.0242692232131958,
  -0.9014149904251099]]

In [22]:
wrapper.embed(["testing"])

[[0.39172840118408203,
  1.1071618795394897,
  0.8399309515953064,
  0.005346053745597601,
  -0.213631734251976,
  -0.19168049097061157,
  -0.09819252043962479,
  -0.7724509835243225,
  -0.33011215925216675,
  -1.0723963975906372,
  -0.00808426458388567,
  -0.4216598570346832,
  0.18245543539524078,
  0.7774981260299683,
  -0.22453688085079193,
  0.5082960724830627,
  0.8072395324707031,
  -0.32681772112846375,
  0.8773772120475769,
  -0.8285756707191467,
  -0.10850708186626434,
  0.4685605764389038,
  0.35311469435691833,
  -0.9684391021728516,
  -0.9024025797843933,
  -1.2777366638183594,
  0.17258375883102417,
  1.0642080307006836,
  0.9749150276184082,
  -0.6136587858200073,
  -1.0525697469711304,
  0.23584699630737305,
  -0.2278098315000534,
  -0.31781333684921265,
  0.1963496208190918,
  -0.36000850796699524,
  0.6943233609199524,
  1.1887900829315186,
  -0.529232919216156,
  0.21239279210567474,
  0.006872887723147869,
  0.07971421629190445,
  -0.3789396286010742,
  0.0139423971

In [23]:
# cached data:
# wrapper.embeddings_training
# wrapper.embeddings_testing
# wrapper.projector
# wrapper.projections_training
# wrapper.projections_testing
# wrapper.salience_maps
# wrapper.clusters
# wrapper.cluster_profiles
# wrapper.cluster_words_freqs
# wrapper.cluster_class_word_sets